In [1]:

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
import h5py


dataset = ''
model = ''
label_bin = ''
plot =''

sets = [dataset, model, label_bin, plot]

In [2]:
data = []
labels = []
imgPaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imgPaths)

for imagePath in imgPaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
data = np.array(data, dtype='float')/255.0
labels = np.array(labels)

(img_train, labels_train, img_test, labels_test) = train_test_split(data, labels, test_size=0.25, random_state=42)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
lb = LabelBinarizer()
#converts labels from int to vectors
#one hot encoding on labels is already done through transform, fit_transform finds all unique classes
labels_train = lb.fit_transform(labels_train)
labels_test = lb.transform(labels_test)

In [ ]:
model = Sequential()
#input is 3072=32x32x3
model.add(Dense(1024, input_shape=(3072,), activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(len(lb.classes_), activation='softmax'))

In [ ]:
sgd = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model_fit = model.fit(img_train, labels_train, validation_data=(img_test, labels_test), epochs=30, batch_size=32)

In [ ]:
predictions = model.predict(img_test, batch_size=32)
print(classification_report(labels_test.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

N = np.arange(0, 30)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, model_fit.history["loss"], label="train_loss")
plt.plot(N, model_fit.history["val_loss"], label="val_loss")
plt.plot(N, model_fit.history["accuracy"], label="train_acc")
plt.plot(N, model_fit.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(sets["plot"])

In [ ]:
model.save(model, save_format='h5')
f = open(label_bin, 'wb')
f.write(pickle.dumps(lb))
f.close()
